In [192]:
# Import packages

import pandas as pd
pd.set_option('display.max_colwidth', -1)
import numpy as np
import twitter


In [142]:
api = twitter.Api(consumer_key = 'FP94BfUftKeygulmcLcVqRNvX',
                 consumer_secret = 'GfZtNuk6Zu6lTOYGWibXI95MjMyks6SSlBEykyLuYe4NEUgGUu',
                 access_token_key = '961088185756393472-FopLzpw7n3CrHhbHoWv8BlnR1mZwhGH',
                 access_token_secret= '62Trp0LVEPvAwoGPH4ov8D4TQe2eEaQETKIfXKCPZX6NN')


In [143]:
print(api.VerifyCredentials())

{"created_at": "Wed Feb 07 04:04:14 +0000 2018", "default_profile": true, "default_profile_image": true, "id": 961088185756393472, "lang": "en", "name": "IUSMM2018", "profile_background_color": "F5F8FA", "profile_image_url": "http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png", "profile_link_color": "1DA1F2", "profile_sidebar_fill_color": "DDEEF6", "profile_text_color": "333333", "screen_name": "iusmm2018"}


In [144]:
# create function to accept search term and tehn fetch the tweets for that term

# def TestData(search_string):
#     try:
#         tweets_fetched = api.GetSearch(search_string, count=100)
#         print("Fetched: "+str(len(tweets_fetched))+" tweets with the term "+search_string+"!!")
#         return [{"text":status.text, "label":None} for status in tweets_fetched]
        
#     except:
#         print("fetch_error: please check")
#         return None
              

# testData = TestData("#Harvey")
              

In [145]:
#testData[0:9]

In [146]:
# Step 2:  To manually annotate the downloaded tweets based on relief terms in the tweets and/or write a function to create a corpus 
#to use as training data.  

def createCorpus(file):
    import csv
    corpus=[]
    with open(file,'rt',encoding='ISO-8859-1') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',',quotechar="\"")
        next(lineReader)
        for row in lineReader:
            corpus.append({"label":row[1],"text":row[0]})
    return corpus


train="labelled.csv"
test = "tweets_harvey_recent.csv"

trainingData = createCorpus(train)
testData = createCorpus(test)



In [147]:
trainingData

[{'label': 'donation',
  'text': 'Give blood. #Harvey. #khou #HouNews https://t.co/RYRbkzl3AO'},
 {'label': 'donation',
  'text': 'Want to help Hurricane #Harvey survivors? Donate blood. Many patients are being relocated to DFW area.... https://t.co/rwGwQhEa6C'},
 {'label': 'donation',
  'text': '#SanAntonio blood donations needed to sustain #Harvey evacuees. Find a station near you https://t.co/gyUe9IdP3y'},
 {'label': 'donation',
  'text': 'Donate money, blood or help the furry friends displaced by #HurricaneHarvey https://t.co/b9kh2KZcOI #Harvey #txwx'},
 {'label': 'donation',
  'text': 'Southside Virginia can help the American Red Cross by donating blood.  Hope to see you on the 11th. #Harvey https://t.co/1Ux5gAv523'},
 {'label': 'donation',
  'text': 'Thinking of everyone in hurricane #Harvey s path. If I can donate blood, please give it. If you can donate clothes,\x89Û_ https://t.co/R2su5hoYOW'},
 {'label': 'donation',
  'text': 'OKCPS donating ahead of #HurricaneHarvey!  Join us

In [148]:
import re
from nltk.tokenize import word_tokenize
from string import punctuation
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk import PorterStemmer, WordNetLemmatizer
from bs4 import BeautifulSoup
from nltk.corpus import stopwords 

# Step 3:  Pre-process the Tweets

class PreProcessTweets:
    def __init__(self):
        self._stopwords=set(stopwords.words('english')+list(punctuation)+['AT_USER','URL'])
              
    def processTweets(self,list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self,tweet):
        # Unescape from HTML
        tweet = BeautifulSoup(tweet, 'lxml').get_text()
        # 3a. Convert to lower case
        tweet=tweet.lower()
        # 3b. Replace links with the word URL 
        tweet=re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)     
        # 3c. Replace @username with "AT_USER"
        tweet=re.sub('@[^\s]+','AT_USER',tweet)
        # 3d. Replace #word with word 
        tweet=re.sub(r'#([^\s]+)',r'\1',tweet)
        # 3e. Replace images with the word IMG 
        tweet = re.sub(r'\bpic.twitter.com\s+', ' ', tweet)
        # 3f Keep only words with letters
        tweet = re.sub('[^a-zA-Z]',' ',tweet)
        # 3g. Remove RT
        tweet = re.sub(r'\brt([\b\s])', ' ', tweet)
        #3h. Apply Lemmatization
        lemmatizer = WordNetLemmatizer()           
        tweet = [lemmatizer.lemmatize(word) for word in tweet.split() if word not in self._stopwords and len(word)>1]  
        #3i. Return tweet words
        return [word for word in tweet]
    
    
tweetProcessor=PreProcessTweets()
ppTrainingData=tweetProcessor.processTweets(trainingData)
ppTestData=tweetProcessor.processTweets(testData)




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jay\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [149]:
# Step 4: Extract features from both training and test data using Support Vector Machines/ Multinomial Naive Bayes

import nltk
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold

#vectorizer = TfidfVectorizer(min_df=5, max_df = 0.8, sublinear_tf=True, use_idf=True,stop_words='english')

vectorizer = TfidfVectorizer()
svmTrainingData=[' '.join(tweet[0]) for tweet in ppTrainingData]



In [150]:
X=vectorizer.fit_transform(svmTrainingData).toarray()
# Create a term document matrix 
vocabulary=vectorizer.get_feature_names()

labels_to_array={"donation":1,"health":2,"volunteering":3,"pets":4,"transportation":5,"relocation":6,"other":7}
labels=[labels_to_array[tweet[1]] for tweet in ppTrainingData]
y=np.array(labels)

# SVM classifier 

SVMClassifier=LinearSVC()
SVMClassifier.fit(X,y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [178]:
# SVM 
SVMResultLabels=[]
for tweet in ppTestData:
    tweet_sentence=' '.join(tweet[0])
    svmFeatures=vectorizer.transform([tweet_sentence]).toarray()
    SVMResultLabels.append(SVMClassifier.predict(svmFeatures)[0])
    



In [179]:
array_to_labels = {value:key for key, value in labels_to_array.items()}

SVMResultLabelsV = [array_to_labels[k] for k in SVMResultLabels]

for i in range(0, len(SVMResultLabelsV)):
    testData[i]['label'] = SVMResultLabelsV[i]

In [186]:
pred_df = pd.DataFrame(testData)

In [194]:
pred_df.head(25)

,label,text
0,relocation,RT @TexasTribune: Gov. Greg Abbott and FEMA are using #HurricaneHarvey to reinvent disaster response. Some say doing that is leaving displaÛ_
1,donation,I know @JJWatt is am amazing man and I live in and Houston and no I don't hear about the money from Harvey but thatÛ_ https://t.co/iK0eAEuYPo
2,donation,RT @CockrellSchool: Could tilting roadways have helped prevent the floods of #HurricaneHarvey? This team of #TexasEngineers is working to iÛ_
3,donation,"RT @AJCGlobal: Last year, HoustonÛªs Congregation @bethyeshurun was flooded by #HurricaneHarvey. Today, in a moving gesture, @azembassyus pÛ_"
4,donation,RT @djriddler: Shout out to the man who created the hoUSton logo that went viral during #HurricaneHarvey @stevenbwallace @kintigo #HandInHaÛ_
5,relocation,Gov. Greg Abbott and FEMA are using #HurricaneHarvey to reinvent disaster response. Some say doing that is leavingÛ_ https://t.co/8189QB5yP6
6,donation,"RT @HoustonChron: Three feet of filthy bayou water rushed our house during #HurricaneHarvey, and we are still wringing ourselves out.\n\nhttpÛ_"
7,donation,RT @brandonformby: .@GregAbbott_TX &amp; FEMA are using #HurricaneHarvey to reinvent disaster response. But reinventing the wheel has left thouÛ_
8,relocation,"RT @brandonformby: Six months after #HurricaneHarvey more than 8,000 Texans remain in hotel rooms. About 5,000 families are getting basic eÛ_"
9,donation,"Your Back""\n \n#Harvey #HurricaneHarvey"


In [ ]:
# Step 5: Train the classifier on the training data

In [ ]:
# Step 6 : Use the classifier to classify the tweets

In [ ]:
# Step 7 : Model Inference & Visualization

In [ ]:
# # Step 4a: build a vocabulary 
# def buildVocabulary(ppTrainingData):
#     all_words=[]
#     for (words,label) in ppTrainingData:
#         all_words.extend(words)
#     wordlist=nltk.FreqDist(all_words)
#     word_features=wordlist.keys()
#     return word_features


# #Step 4b : Represent each tweet with presence/absence of the relief terms in the tweet 
# def extract_features(tweet):
#     tweet_words=set(tweet)
#     features={}
#     for word in word_features:
#         features['contains(%s)' % word]=(word in tweet_words)
#     return features 

# # Extract the features and train the classifier 

# word_features = buildVocabulary(ppTrainingData)
# trainingFeatures=nltk.classify.apply_features(extract_features,ppTrainingData)

# #NBayesClassifier=nltk.NaiveBayesClassifier.train(trainingFeatures)